In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.utils import resample
import seaborn as sns

sns.set_theme(style='whitegrid')
np.random.seed(3)

In [2]:
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

In [4]:
n_boostraps = 100
degree = 5
noise = 0.1

# Make data
n = 100
x = np.linspace(0, 1, n)
y = np.linspace(0, 1, n)
x, y = np.meshgrid(x,y)

# we unravel the values to change the shape from (100, 100) to (10000, )
x = x.ravel()
y = y.ravel()

k = .01 # noise coefficient 
z = FrankeFunction(x, y) # + k*np.random.randn(n^2)

X = np.column_stack((x,y))

# Hold out some test data that is never used in training.
X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2, random_state=3)


pipeline = make_pipeline(
        PolynomialFeatures(degree=degree),
        StandardScaler(),
        Lasso(alpha=1e-4, max_iter=12_000))

# The following (m x n_bootstraps) matrix holds the column vectors y_pred for each bootstrap iteration.
z_pred = np.empty((z_test.shape[0], n_boostraps))
for i in range(n_boostraps):
    x_, y_ = resample(X_train, z_train) # bootstrap from sklearn.utils

    # Evaluate the new model on the same test data each time
    z_pred[:, i] = pipeline.fit(x_, y_).predict(X_test).ravel()

z_test = z_test.reshape((2000, 1))

# Note 2: The use of keepdims=True is important in the calculation of bias as this
# maintains the column vector form. Dropping this yields very unexpected results.
error = np.mean( np.mean((z_test - z_pred)**2, axis=1, keepdims=True) )
bias = np.mean( (z_test - np.mean(z_pred, axis=1, keepdims=True))**2 )
variance = np.mean( np.var(z_pred, axis=1, keepdims=True) )
print('Error:', error)
print('Bias^2:', bias)
print('Var:', variance)
print(f'{error} >= {bias} + {variance} = {bias+variance}')
print(f'{error - (bias+variance)}')

c:\Users\jonas\anaconda3\envs\fysstk3155\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.782e+00, tolerance: 6.625e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\jonas\anaconda3\envs\fysstk3155\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.248e+00, tolerance: 6.627e-02
  model = cd_fast.enet_coordinate_descent(
c:\Users\jonas\anaconda3\envs\fysstk3155\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing 

Error: 0.0050271121281015
Bias^2: 0.005017686073177854
Var: 9.426054923646263e-06
0.0050271121281015 >= 0.005017686073177854 + 9.426054923646263e-06 = 0.0050271121281015
0.0


c:\Users\jonas\anaconda3\envs\fysstk3155\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.171e+00, tolerance: 6.427e-02
  model = cd_fast.enet_coordinate_descent(
